In [3]:
#image processing project
import cv2
import numpy as np
import os
# Load the class labels MobileNet SSD was trained to detect
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant",
           "sheep", "sofa", "train", "tvmonitor"]

# Provide the absolute paths to the model files
prototxt_path = os.path.abspath(r"C:\Users\2004a\notebook\Projects\image recognition system\MobileNetSSD_deploy.prototxt")
model_path = os.path.abspath(r"C:\Users\2004a\notebook\Projects\image recognition system\MobileNetSSD_deploy.caffemodel")

# Load the serialized model from disk
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

def detect_objects(image_path):
    # Load the input image
    image = cv2.imread(image_path)
    (h, w) = image.shape[:2]

    # Preprocess the image: resize and mean subtraction
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    # Pass the blob through the network and obtain the detections and predictions
    net.setInput(blob)
    detections = net.forward()

    # Loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # Extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections by ensuring the `confidence` is greater than a minimum threshold
        if confidence > 0.2:
            # Extract the index of the class label from the `detections`
            idx = int(detections[0, 0, i, 1])
            label = CLASSES[idx]

            # Compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Draw the prediction on the image
            cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, f"{label}: {confidence:.2f}", (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the output image
    cv2.imshow("Output", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
detect_objects("rose.jpg")
